In [ ]:
import os
import pandas as pd
import re
import pymysql
import cn2an
import json
from IPython.display import display
from datetime import datetime
from pathlib import Path

In [ ]:
# 左侧的查询按钮
def execute_sql(conn) -> None:
    # ——————————————————————需要从界面中获取 user_sql————————————————
    # user_sql = "SELECT * FROM documents LIMIT 5" # 这只是个例子
    # ——————————————————————需要从界面中获取 user_sql————————————————

    try:
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
            cursor.execute(user_sql)
            result = cursor.fetchall()
            cleaned_result = [
                {k: str(v) if v is not None else "" for k, v in row.items()}
                for row in result
            ]
            print(json.dumps(cleaned_result, ensure_ascii=False, indent=4))
    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 文献页面（加载内容按钮）
# 目前只有简单的输出 print，若需要把这个结果带动main函数中，直接改成
# return result 即可
def load_document_data(conn) -> None:
    # doc_id = 1  # 示例文献 ID（假设由前端传入）
    try:
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
            sql = "SELECT * FROM documents WHERE doc_id = %s"
            cursor.execute(sql, (doc_id,))
            result = cursor.fetchone()
            if result:
                cleaned_result = {k: str(v) if v is not None else "" for k, v in result.items()}
                print(json.dumps(cleaned_result, ensure_ascii=False, indent=4))
            else:
                print(json.dumps({"status": "error", "message": "未找到对应文献。"}, ensure_ascii=False, indent=4))
    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 文献页面（修改按钮）

def update_document_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————
    # data = {
    #     "doc_id": 1,
    #     "doc_title": "永乐大典",
    #     "category_type": 1,
    #     "doc_specific_category": "谱牒",
    #     "doc_origin_id": "567",
    #     "doc_style": "类事",
    #     "doc_type": 1,
    #     "doc_image": "/images/cover.jpg",
    #     "doc_theme": "天",
    #     "dynasty": "明",
    #     "compilation_time": "1403",
    #     "printing_time": "1408",
    #     "publication_time": "1410",
    #     "completeness": "全本",
    #     "source": "系统录入"
    # }
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        fields = [k for k in data if k != "doc_id"]
        set_clause = ", ".join([f"{k} = %s" for k in fields])
        values = [data[k] for k in fields] + [data["doc_id"]]

        sql = f"UPDATE documents SET {set_clause} WHERE doc_id = %s"

        with conn.cursor() as cursor:
            cursor.execute(sql, values)
            print(json.dumps({"status": "success", "updated_rows": cursor.rowcount}, ensure_ascii=False, indent=4))

    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 文献页面（插入按钮）
def insert_document_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————
    # data = {
    #     "doc_title": "新撰类书",
    #     "category_type": 1,
    #     "doc_specific_category": "谱牒",
    #     "doc_origin_id": "123",
    #     "doc_style": "类事",
    #     "doc_type": 1,
    #     "doc_image": "/images/example.jpg",
    #     "doc_theme": "天",
    #     "dynasty": "宋",
    #     "compilation_time": "1070",
    #     "printing_time": "1080",
    #     "publication_time": "1085",
    #     "completeness": "全本",
    #     "source": "用户输入"
    # }
    # ——————————————————————需要从界面中获取字段值————————————————
    try:
        fields = list(data.keys())
        placeholders = ", ".join(["%s"] * len(fields))
        columns = ", ".join(fields)
        values = [data[k] for k in fields]

        sql = f"INSERT INTO documents ({columns}) VALUES ({placeholders})"

        with conn.cursor() as cursor:
            cursor.execute(sql, values)
            print(json.dumps({"status": "success", "inserted_id": cursor.lastrowid}, ensure_ascii=False, indent=4))

    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 编纂者页面（加载内容按钮）
def load_author_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————

    # author_id = 1  # 示例值
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
            sql = "SELECT * FROM authors WHERE author_id = %s"
            cursor.execute(sql, (author_id,))
            result = cursor.fetchone()
            if result:
                cleaned_result = {k: str(v) if v is not None else "" for k, v in result.items()}
                print(json.dumps(cleaned_result, ensure_ascii=False, indent=4))
            else:
                print(json.dumps({"status": "error", "message": "未找到对应编纂者。"}, ensure_ascii=False, indent=4))
    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 编纂者页面（修改按钮）

def update_author_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————
    # data = {
    #     "author_id": 2,
    #     "author_name": "李四",
    #     "author_org": "国家图书馆"
    # }
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        fields = [k for k in data if k != "author_id"]
        set_clause = ", ".join([f"{k} = %s" for k in fields])
        values = [data[k] for k in fields] + [data["author_id"]]

        sql = f"UPDATE authors SET {set_clause} WHERE author_id = %s"

        with conn.cursor() as cursor:
            cursor.execute(sql, values)
            print(json.dumps({"status": "success", "updated_rows": cursor.rowcount}, ensure_ascii=False, indent=4))

    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 编纂者页面（插入按钮）
def insert_author_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————
    # data = {
    #     "author_name": "王五",
    #     "author_org": "明朝翰林院"
    # }
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        fields = list(data.keys())
        placeholders = ", ".join(["%s"] * len(fields))
        columns = ", ".join(fields)
        values = [data[k] for k in fields]

        sql = f"INSERT INTO authors ({columns}) VALUES ({placeholders})"

        with conn.cursor() as cursor:
            cursor.execute(sql, values)
            print(json.dumps({"status": "success", "inserted_id": cursor.lastrowid}, ensure_ascii=False, indent=4))

    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 文献——编纂者关联（加载数据按钮）
def load_document_author_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————

    # da_id = 1  # 示例关联ID
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
            sql = "SELECT * FROM document_author_links WHERE da_id = %s"
            cursor.execute(sql, (da_id,))
            result = cursor.fetchone()
            if result:
                cleaned_result = {k: str(v) if v is not None else "" for k, v in result.items()}
                print(json.dumps(cleaned_result, ensure_ascii=False, indent=4))
            else:
                print(json.dumps({"status": "error", "message": "未找到对应关联记录。"}, ensure_ascii=False, indent=4))
    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 文献——编纂者关联（修改按钮）
def update_document_author_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————

    # data = {
    #     "da_id": 1,
    #     "doc_id": 3,
    #     "author_id": 2,
    #     "role": "主编"
    # }
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        fields = [k for k in data if k != "da_id"]
        set_clause = ", ".join([f"{k} = %s" for k in fields])
        values = [data[k] for k in fields] + [data["da_id"]]

        sql = f"UPDATE document_author_links SET {set_clause} WHERE da_id = %s"

        with conn.cursor() as cursor:
            cursor.execute(sql, values)
            print(json.dumps({"status": "success", "updated_rows": cursor.rowcount}, ensure_ascii=False, indent=4))

    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 文献——编纂者关联（插入按钮）
def insert_document_author_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————
    # data = {
    #     "doc_id": 3,
    #     "author_id": 2,
    #     "role": "校对"
    # }
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        fields = list(data.keys())
        placeholders = ", ".join(["%s"] * len(fields))
        columns = ", ".join(fields)
        values = [data[k] for k in fields]

        sql = f"INSERT INTO document_author_links ({columns}) VALUES ({placeholders})"

        with conn.cursor() as cursor:
            cursor.execute(sql, values)
            print(json.dumps({"status": "success", "inserted_id": cursor.lastrowid}, ensure_ascii=False, indent=4))

    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 标题（加载数据按钮）
def load_title_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————

    # title_id = 1  
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
            sql = "SELECT * FROM titles WHERE title_id = %s"
            cursor.execute(sql, (title_id,))
            result = cursor.fetchone()
            if result:
                cleaned_result = {k: str(v) if v is not None else "" for k, v in result.items()}
                print(json.dumps(cleaned_result, ensure_ascii=False, indent=4))
            else:
                print(json.dumps({"status": "error", "message": "找不到对应标题。"}, ensure_ascii=False, indent=4))
    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 标题（修改按钮）
def update_title_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————

    # data = {
    #     "title_id": 1,
    #     "doc_id": 3,
    #     "title_name": "修订天地人",
    #     "title_level": "h2",
    #     "parent_id": 5,
    #     "title_order": 2
    # }
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        fields = [k for k in data if k != "title_id"]
        set_clause = ", ".join([f"{k} = %s" for k in fields])
        values = [data[k] for k in fields] + [data["title_id"]]

        sql = f"UPDATE titles SET {set_clause} WHERE title_id = %s"

        with conn.cursor() as cursor:
            cursor.execute(sql, values)
            print(json.dumps({"status": "success", "updated_rows": cursor.rowcount}, ensure_ascii=False, indent=4))

    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 标题（插入按钮）
def load_title_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————

    # title_id = 1 
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
            sql = "SELECT * FROM titles WHERE title_id = %s"
            cursor.execute(sql, (title_id,))
            result = cursor.fetchone()
            if result:
                cleaned_result = {k: str(v) if v is not None else "" for k, v in result.items()}
                print(json.dumps(cleaned_result, ensure_ascii=False, indent=4))
            else:
                print(json.dumps({"status": "error", "message": "找不到对应标题。"}, ensure_ascii=False, indent=4))
    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 标题（加载数据按钮）
def load_page_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————

    # page_id = 1
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
            sql = "SELECT * FROM pages WHERE page_id = %s"
            cursor.execute(sql, (page_id,))
            result = cursor.fetchone()
            if result:
                cleaned_result = {k: str(v) if v is not None else "" for k, v in result.items()}
                print(json.dumps(cleaned_result, ensure_ascii=False, indent=4))
            else:
                print(json.dumps({"status": "error", "message": "未找到对应页码。"}, ensure_ascii=False, indent=4))
    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 标题（修改数据按钮）

def update_page_data(conn) -> None:
    # ——————————————————————需要从界面中获取字段值————————————————

    # data = {
    #     "page_id": 1,
    #     "doc_id": 3,
    #     "full_text_id_list": "12,13,14",
    #     "page_number": 23,
    #     "page_type": "A",
    #     "page_image": "/images/pages/page23.jpg",
    #     "title_id": 5
    # }
    # ——————————————————————需要从界面中获取字段值————————————————

    try:
        fields = [k for k in data if k != "page_id"]
        set_clause = ", ".join([f"{k} = %s" for k in fields])
        values = [data[k] for k in fields] + [data["page_id"]]

        sql = f"UPDATE pages SET {set_clause} WHERE page_id = %s"

        with conn.cursor() as cursor:
            cursor.execute(sql, values)
            print(json.dumps({"status": "success", "updated_rows": cursor.rowcount}, ensure_ascii=False, indent=4))

    except Exception as e:
        print(json.dumps({"status": "error", "message": str(e)}, ensure_ascii=False, indent=4))


In [ ]:
# 这是有关全文插入的函数，是run_all_insert_steps函数的一部分
def insert_documents_from_result(result_file_path, connection):
    with open(result_file_path, 'r', encoding='utf-8') as f:
        result = json.load(f)

    try:
        with connection.cursor() as cursor:
            for item in result:
                if item.get("text_type") == "引书":
                    # 整理书名
                    book_title = item.get('full_text').strip()
                    book_title_wrapped = f"《{book_title}》"
                    doc_id_to_add = str(item.get("doc_id"))

                    # 使用 MATCH AGAINST 查询文献是否已存在（包含 doc_origin_id）
                    cursor.execute(
                        """
                        SELECT doc_id, doc_origin_id FROM Documents
                        WHERE MATCH(doc_title) AGAINST (%s IN BOOLEAN MODE)
                        """,
                        (f'"{book_title}"',)
                    )
                    existing = cursor.fetchone()

                    if existing:
                        this_doc_id = existing["doc_id"]
                        this_origin = existing.get("doc_origin_id")

                        # 将 origin 处理为列表，检查是否已包含 doc_id_to_add
                        if this_origin and this_origin.strip():
                            origin_list = [x.strip() for x in this_origin.split(',') if x.strip()]
                            if doc_id_to_add in origin_list:
                                print(f"{book_title_wrapped} ➤ 已存在，origin_id 中已包含 {doc_id_to_add}")
                            else:
                                origin_list.append(doc_id_to_add)
                                new_origin = ','.join(origin_list)
                                cursor.execute(
                                    """
                                    UPDATE Documents
                                    SET doc_origin_id = %s
                                    WHERE doc_id = %s
                                    """,
                                    (new_origin, this_doc_id)
                                )
                                print(f"{book_title_wrapped} ➤ 已存在，已将 {doc_id_to_add} 添加到 origin_id")
                        else:
                            # 如果 origin_id 是 None 或空字符串，则重新设置
                            cursor.execute(
                                """
                                UPDATE Documents
                                SET doc_origin_id = %s
                                WHERE doc_id = %s
                                """,
                                (doc_id_to_add, this_doc_id)
                            )
                            print(f"{book_title_wrapped} ➤ 已存在，origin_id 设置为 {doc_id_to_add}")

                    else:
                        # 插入新文献记录
                        cursor.execute(
                            """
                            INSERT INTO Documents (doc_title, doc_type, doc_origin_id)
                            VALUES (%s, %s, %s)
                            """,
                            (book_title_wrapped, False, doc_id_to_add)
                        )
                        print(f"{book_title_wrapped} ➤ 新文献插入完成（origin_id={doc_id_to_add}）")
    except Exception as e:
        print(f"发生错误: {e}")

In [ ]:
# 这是有关全文插入的函数，是run_all_insert_steps函数的一部分
def insert_full_text_from_result(file_path, conn):
    cursor = conn.cursor()

    # 从文件中加载 result 数据
    with open(file_path, 'r', encoding='utf-8') as f:
        result = json.load(f)

    # 插入 SQL 语句
    insert_query = '''
    INSERT INTO full_text_1
    (full_text, full_text_order, title_level, title_id, text_type, related_id, quote_loc, doc_id, page_number, page_type)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''

    data_to_insert = []
    for entry in result:
        data_tuple = (
            entry.get('full_text'),
            entry.get('full_text_order'),
            entry.get('title_level'),
            entry.get('title_id'),
            entry.get('text_type'),
            entry.get('related_id', None),
            entry.get('quote_loc', None),
            entry.get('doc_id'),
            entry.get('page_number', None),
            entry.get('page_type', None)
        )
        data_to_insert.append(data_tuple)

    # 执行插入操作
    try:
        cursor.executemany(insert_query, data_to_insert)
        print(f"{cursor.rowcount} 条数据已成功插入 full_text_1 表。")
    except Exception as e:
        print(f"发生错误: {e}")

In [ ]:
# 这是有关全文插入的函数，是run_all_insert_steps函数的一部分
def update_full_text_relationships_from_result(file_path, conn):
    # 从文件中读取 title_id
    with open(file_path, 'r', encoding='utf-8') as f:
        result = json.load(f)
    title_id = result[0].get("title_id")

    try:
        with conn.cursor(pymysql.cursors.DictCursor) as cursor:
            # 1. 查询 Full_Text_1 中该 title_id 的所有数据
            cursor.execute("""
                SELECT full_text_id, text_type
                FROM Full_Text_1
                WHERE title_id = %s
            """, (title_id,))
            rows = cursor.fetchall()
            print(f"共获取数据 {len(rows)} 条\n")

            # 2. 跟踪最后出现的 引书 和 引文 的 ID
            last_yinshu_id = None
            last_yinwen_id = None
            update_count = 0

            # 3. 遍历数据并设置 related_id
            for row in rows:
                full_text_id = row['full_text_id']
                text_type = row['text_type']
                related_id = None

                if text_type == '引书':
                    last_yinshu_id = full_text_id
                    print(f"发现引书 → ID {full_text_id}（related_id 不变）")
                    continue

                elif text_type == '引文':
                    if last_yinshu_id is not None:
                        related_id = last_yinshu_id
                        last_yinwen_id = full_text_id
                        print(f"引文 → 关联到引书 ID {related_id}")
                    else:
                        print(f"警告：没有引书情况下出现引文！ID {full_text_id}")
                        continue

                elif text_type == '注疏':
                    if last_yinwen_id is not None:
                        related_id = last_yinwen_id
                        print(f"注疏 → 关联到引文 ID {related_id}")
                    else:
                        print(f"警告：没有引文情况下出现注疏！ID {full_text_id}")
                        continue

                if related_id is not None:
                    cursor.execute("""
                        UPDATE Full_Text_1
                        SET related_id = %s
                        WHERE full_text_id = %s
                    """, (related_id, full_text_id))
                    update_count += 1

            print(f"\n共更新 {update_count} 条 related_id 字段。")
    except Exception as e:
        print(f"发生错误: {e}")
        
    print("所有关联设置已完成。")

In [ ]:
# 这是有关全文插入的函数，是run_all_insert_steps函数的一部分
def insert_pages_from_result(file_path, conn):
    cursor = conn.cursor(pymysql.cursors.DictCursor)

    # 从文件中读取 title_id
    with open(file_path, 'r', encoding='utf-8') as f:
        result = json.load(f)
    title_id = result[0].get("title_id")

    # 查询 full_text_1 中各页的 full_text_id 列表
    select_query = '''
    SELECT 
        doc_id,
        title_id,
        page_number,
        page_type,
        GROUP_CONCAT(full_text_id ORDER BY full_text_id ASC) AS full_text_id_list
    FROM 
        full_text_1
    WHERE
        title_id = %s
    GROUP BY
        doc_id, title_id, page_number, page_type
    ORDER BY
        doc_id ASC, title_id ASC, page_number ASC, page_type ASC;
    '''

    cursor.execute(select_query, (title_id,))
    rows = cursor.fetchall()

    # 构建插入 pages 表的数据
    pages_data = []

    for row in rows:
        pages_data.append((
            row['doc_id'],
            row['full_text_id_list'],
            row['page_number'],
            row['page_type'],
            None,              # page_image 暂无
            datetime.now(),    # create_time
            row['title_id']
        ))

        print(f"doc_id {row['doc_id']}, page_number {row['page_number']}, page_type {row['page_type']}, title_id {row['title_id']}, 包含 full_text_id: {row['full_text_id_list']}")

    # 插入到 Pages 表中
    insert_query = '''
    INSERT INTO pages
    (doc_id, full_text_id_list, page_number, page_type, page_image, create_time, title_id)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    '''

    try:
        cursor.executemany(insert_query, pages_data)
        print(f"{cursor.rowcount} 条数据已插入 pages 表。")
    except Exception as e:
        print(f"发生错误: {e}")

In [ ]:
# 这是有关全文插入的函数，调用它，从引书目录，全文，全文关联，页码都会自动插入
def run_all_insert_steps(file_path, conn):
    insert_documents_from_result(file_path, conn)
    insert_full_text_from_result(file_path, conn)
    update_full_text_relationships_from_result(file_path, conn)
    insert_pages_from_result(file_path, conn)
    return

In [ ]:
# ——————————————————————————MAIN——————————————————————————————————
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='park1947',
    database='leishu_yongle',
    charset='utf8mb4',
    autocommit=False,  # 不能直接提交
    cursorclass=pymysql.cursors.DictCursor
)

try:

    # 这是数据标注页面的全部函数
    # execute_sql(conn) # 它是左侧的，基于SQL语句查询内容的函数
    # load_document_data(conn) # 加载数据按钮
    # update_document_data(conn) # 修改按钮
    # insert_document_data(conn) # 插入按钮

    # load_author_data(conn) # 加载数据按钮
    # update_author_data(conn) # 修改按钮
    # insert_author_data(conn) # 插入按钮

    # load_document_author_data(conn) # 加载数据按钮
    # update_document_author_data(conn) # 修改按钮
    # insert_document_author_data(conn) # 插入按钮
    
    # load_title_data(conn) # 加载数据按钮
    # update_title_data(conn) # 修改按钮
    # insert_title_data(conn) # 插入按钮
    
    # load_page_data(conn) # 加载数据按钮
    # update_page_data(conn) # 修改按钮
    # 页码本身没有插入功能，全文自动插入会自动插入页码，用户只能修改

    # 全文插入时，要求用户输入“识别结果txt文件”，这样才能执行 run_all_insert_steps

    # ——————————————————————需要获取用户的 txt文件————————————————
    # file_path = r'C:\Users\cuiwe\Desktop\212\从1到14识别结果.txt'
    # ——————————————————————需要获取用户的 txt文件————————————————


    # run_all_insert_steps(file_path, conn)



    # ——————————————————————这个部分需要改————————————————

    # conn.begin()  #开始事物

    # 调用函数部分
    # 调用函数部分
    # 调用函数部分
    # 调用函数部分
    # 调用函数部分
    # 调用函数部分
    # 调用函数部分

    # if 点击“提交” -> conn.commit()
    
    # ——————————————————————这个部分需要改————————————————


except Exception as e:
    conn.rollback()
finally:
    conn.close()


IndentationError: expected an indented block after 'try' statement on line 13 (1787464684.py, line 53)